In [1]:
import numpy as np
from scipy import stats
from joblib import Parallel, delayed

class CART:
    def __init__(self, max_depth=None, min_samples_split=2, mode="regression"):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.mode = mode
        self.tree = None
    
    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def predict(self, X):
        return np.array([self._predict_row(row, self.tree) for row in X])
    
    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        unique_values = np.unique(y)
        if len(unique_values) == 1 or (self.max_depth and depth == self.max_depth) or num_samples < self.min_samples_split:
            if self.mode == 'regression':
                return np.mean(y)
            else:
                return stats.mode(y, keepdims=False).mode
        
        best_split = None
        best_score = float('inf')

        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold
                left_y = y[left_mask]
                right_y = y[~left_mask]

                if len(left_y) == 0 or len(right_y) == 0:
                    continue

                if self.mode == 'regression':
                    left_rss = np.sum((left_y - left_y.mean()) ** 2)
                    right_rss = np.sum((right_y - right_y.mean()) ** 2)
                    score = (left_rss + right_rss) / num_samples
                else:
                    left_gini = self._gini(left_y)
                    right_gini = self._gini(right_y)
                    score = (len(left_y) * left_gini + len(right_y) * right_gini) / num_samples

                if score < best_score:
                    best_score = score
                    best_left_mask = left_mask
                    best_split = (feature_idx, threshold)

        if best_split is None:
            if self.mode == 'regression':
                return np.mean(y)
            else:
                return stats.mode(y, keepdims=False).mode

        left_tree = self._build_tree(X[best_left_mask], y[best_left_mask], depth + 1)
        right_tree = self._build_tree(X[~best_left_mask], y[~best_left_mask], depth + 1)
        
        return {
            'best_split': best_split,
            'left': left_tree,
            'right': right_tree
        }

    def _predict_row(self, row, tree):
        if isinstance(tree, dict):
            feature, threshold = tree['best_split']
            if row[feature] <= threshold:
                return self._predict_row(row, tree['left'])
            else:
                return self._predict_row(row, tree['right'])
        else:
            return tree
    
    def _gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        p = counts / len(y)
        return 1 - np.sum(p ** 2)


In [2]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeClassifier, DecisionTreeRegressor
class RandomForest:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, max_features='sqrt', oob_score=False, mode="regression", random_state=0, n_jobs=-1):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.oob_score = oob_score
        self.oob_score_ = None  # 用于存储OOB得分
        self.mode = mode
        self.random_state = random_state
        self.n_jobs = n_jobs

        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.estimators = None

    def fit(self, X, y):
        np.random.seed(self.random_state)
        if not isinstance(y, np.ndarray):
            y = np.array(y)
        
        # 并行训练每棵树
        self.estimators = Parallel(n_jobs=self.n_jobs)(
            delayed(self._train_tree)(X, y, i)
            for i in range(self.n_estimators)
        )
        
        # 计算袋外得分
        if self.oob_score:
            self._cal_oob_score(X, y)
    
    def _train_tree(self, X, y, i):
        np.random.seed(self.random_state + i)
        num_samples, num_features = X.shape
        
        # boostrap
        sample_idx = np.random.choice(num_samples, size=num_samples, replace=True)
        X_k, y_k = X[sample_idx], y[sample_idx]
        
        if self.max_features == 'sqrt':
            feature_idx = np.random.choice(num_features, size=int(np.sqrt(num_features)), replace=False)
        elif self.max_features == 'log2':
            feature_idx = np.random.choice(num_features, size=int(np.log2(num_features)), replace=False)
        else:
            feature_idx = np.random.choice(num_features, size=num_features, replace=False)
        
        estimator = CART(self.max_depth, self.min_samples_split, self.mode)
        estimator.fit(X_k[:, feature_idx], y_k)
        return (estimator, feature_idx, sample_idx)
    
    def predict(self, X):
        y_pred = []
        for estimator, feature_idx, _ in self.estimators:
            y_pred.append(estimator.predict(X[:, feature_idx]))
        y_pred = np.array(y_pred).T
        self._y_pred = y_pred
        if self.mode == 'classification':
            y_pred = stats.mode(y_pred, axis=1, keepdims=False).mode
        elif self.mode == 'regression':
            y_pred = np.mean(y_pred, axis=1)
        return y_pred
    
    def _cal_oob_score(self, X, y):
        num_samples, num_features = X.shape
        if self.mode == "regression":
            oob_predictions = np.zeros(num_samples)  # 存储袋外样本的预测结果
        else:
            oob_predictions = [[] for _ in range(num_samples)]
        oob_count = np.zeros(num_samples)  # 记录每个样本被多少棵树预测过
        for estimator, feature_idx, sample_idx in self.estimators:
            oob_idx = np.setdiff1d(np.arange(num_samples), sample_idx)
            # 对袋外样本进行预测
            for idx in oob_idx:
                oob_count[idx] += 1
                y_pred_oob = estimator._predict_row(X[idx, feature_idx], estimator.tree)

                if self.mode == "regression":
                    oob_predictions[idx] += y_pred_oob  # 回归任务是累加
                else:
                    oob_predictions[idx].append(y_pred_oob)  # 分类时采用投票
        mask = oob_count > 0
        if self.mode == "regression":
            # 计算R方
            oob_predictions = oob_predictions[mask] / oob_count[mask]  # 对每个样本的袋外预测结果进行平均
            self.oob_score_ = 1 - np.sum((oob_predictions - y[mask]) ** 2) / np.sum((y[mask] - y[mask].mean()) ** 2)
        else:
            # 计算分类准确率
            oob_predictions = np.array([stats.mode(predictions, keepdims=False).mode for predictions in oob_predictions if predictions])
            self.oob_score_ = np.mean(oob_predictions == y[mask])  # 计算准确率

In [4]:
from sklearn.datasets import load_iris, make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

# 加载Iris数据集
iris = load_iris()
X, y = iris.data, iris.target

# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用你实现的RandomForest进行训练
rf_custom = RandomForest(n_estimators=10, max_depth=3, mode="classification", random_state=42, oob_score=True, n_jobs=2)
rf_custom.fit(X_train, y_train)
y_pred_custom = rf_custom.predict(X_test)

# 使用sklearn的RandomForestClassifier进行训练
rf_sklearn = RandomForestClassifier(n_estimators=10, max_depth=3, random_state=42, oob_score=True)
rf_sklearn.fit(X_train, y_train)
y_pred_sklearn = rf_sklearn.predict(X_test)

# 对比预测结果
accuracy_custom = accuracy_score(y_test, y_pred_custom)
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)

print("自实现 RandomForest 分类准确率:", accuracy_custom)
print('oob score:', rf_custom.oob_score_)
print("sklearn RandomForest 分类准确率:", accuracy_sklearn)
print('sklearn oob score:', rf_sklearn.oob_score_)


自实现 RandomForest 分类准确率: 1.0
oob score: 0.875
sklearn RandomForest 分类准确率: 1.0
sklearn oob score: 0.9416666666666667


In [8]:
rf_custom.estimators

[(<__main__.CART at 0x20ee710aef0>,
  array([1, 2]),
  array([102,  51,  92,  14, 106,  71,  60,  20, 102,  82,  86,  74,  74,
          87, 116,  99, 103,  23,   2,  21,  52,   1,  87, 107,  29,  37,
           1,  63,  59,  20,  32,  75,  57,  21, 107,  88,  48,  90,  58,
          41,  91,  59,  79,  14,  61,  61,  46,  61,  50, 107,  54, 115,
          63,   2, 100,  50,   6,  20,  72,  38,  17,   3,  88,  59,  13,
         113,   8,  89,  52,   1,  83,  91, 110,  59,  70,  43,   7,  46,
          34,  77,  80,  35,  49, 103,   3,   1,   5,  53, 105,   3,  53,
          92,  62,  17,  89,  43,  33,  73,  61,  99,  13,  94,  47,  14,
          71,  77,  86,  61,  39,  84,  79, 108,  81, 110,  52,  23,  25,
          88,  59, 108])),
 (<__main__.CART at 0x20ee7859c30>,
  array([3, 1]),
  array([ 68,  64,  49,  21,  58,  16,  51,  17,  59,  91,  66, 110,  30,
          87,  64,  35,  50,  75,  89,  65, 111,  34,  75,  66,  99, 117,
           4,  36, 112, 117,  76,  88,  47,  32,  74,

In [7]:
RANDOM_STATE = 42
# 生成回归数据集
X, y = make_regression(n_samples=100, n_features=5, noise=0.1, random_state=RANDOM_STATE)

# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

# 使用你实现的RandomForest进行训练
rf_custom = RandomForest(n_estimators=100, 
                         max_depth=3, 
                         mode="regression", 
                         random_state=RANDOM_STATE, 
                         max_features='sqrt',
                         oob_score=True,
                         n_jobs=-1)
rf_custom.fit(X_train, y_train)
y_pred_custom = rf_custom.predict(X_test)

# 使用sklearn的RandomForestRegressor进行训练
rf_sklearn = RandomForestRegressor(n_estimators=10, 
                                max_depth=3, 
                                random_state=RANDOM_STATE,
                                max_features='sqrt',
                                oob_score=True)
rf_sklearn.fit(X_train, y_train)
y_pred_sklearn = rf_sklearn.predict(X_test)

# 对比预测结果
mse_custom = mean_squared_error(y_test, y_pred_custom)
mse_sklearn = mean_squared_error(y_test, y_pred_sklearn)

print("自实现 RandomForest 回归 MSE:", mse_custom)
print('自实现 oob score:', rf_custom.oob_score_)
print("sklearn RandomForest 回归 MSE:", mse_sklearn)
print('sklearn oob score:', rf_sklearn.oob_score_)

自实现 RandomForest 回归 MSE: 9062.770669329513
自实现 oob score: 0.4999829807854419
sklearn RandomForest 回归 MSE: 6500.3308179788455
sklearn oob score: 0.4319055562255957


In [15]:
np.set_printoptions(precision=2, suppress=True)
rf_custom.y_pred

array([[ 124.15,  185.1 ,  133.92,  143.8 ,   15.7 ,  132.54,  150.33,
         151.49,  140.75,  183.31],
       [ -40.28, -136.48,   12.6 ,  -89.34, -181.74,  -56.52, -217.28,
        -262.02,  -91.14, -168.82],
       [  93.01,  -12.48,   50.56,   87.65,   84.8 ,   85.43,  -43.82,
          47.82,   45.74,   75.78],
       [ 124.15,  185.1 ,  133.92,  143.8 ,  -15.5 ,  132.54,   82.67,
        -148.34,  237.32, -168.82],
       [ 124.15,  185.1 ,  133.92,  143.8 ,   15.7 ,  132.54,  150.33,
         -16.8 ,  237.32,  -89.56],
       [ -40.28,  -12.48,   50.56,   87.65,   15.7 ,  -56.52,  176.52,
         151.49,   45.74,  183.31],
       [ 124.15,  185.1 ,  133.92,  143.8 ,   15.7 ,  132.54,  150.33,
         -16.8 ,  140.75,   60.35],
       [   0.2 , -136.48, -217.84,  -79.92,   15.7 ,   85.43,  -43.82,
          47.82,  -91.14,   75.78],
       [ 359.27,   72.11,   27.69,  143.8 ,  -15.5 ,  132.54,   82.67,
          47.82,   45.74,   70.16],
       [ -40.28, -136.48, -217.84,  -

In [ ]:
from multiprocessing import Value
class MyClass:
    def __init__(self, x):
        self.x = x
        self.loop = 10

    def fit(self):

        Parallel(n_jobs=2)(delayed(self.fun)(i) for i in range(self.loop))

    def fun(self, i):
        self.x += 1
        print(i)

a = MyClass(0)
a.fit()
print(a.x)  # 输出应该是 10

0


In [37]:
from joblib import Parallel, delayed
from multiprocessing import Value

class MyClass:
    def __init__(self, x):
        self.x = Value('i', x)  # 创建一个共享变量 'x'
        self.loop = 10

    def fit(self):
        # 使用 Parallel 来并行化调用 fun 方法
        Parallel(n_jobs=2)(delayed(self.fun)(i) for i in range(self.loop))

    def fun(self, i):
        # 更新共享变量 x
        with self.x.get_lock():  # 使用锁来保证对共享变量的安全访问
            self.x.value += 1
        print(i)

a = MyClass(0)
a.fit()
print(a.x.value)  # 输出应该是 10

PicklingError: Could not pickle the task to send it to the workers.